


PageRank is an algorithm used by Google to rank web pages in the search engine results. PageRank is an iterative algorithm which is a perfect fit for Spark. In this question, you will implement PageRank algorithm with PySpark by completing the following codes. 

In [13]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

tar: spark-2.4.5-bin-hadoop2.7.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

!wget -q https://www.dropbox.com/s/dylx9471funotis/network.txt

tar: spark-2.4.5-bin-hadoop2.7.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


Exception: ignored

First, we load the sample network file in Spark RDD, as below. 

This file has format: **source_page target_page**. Below is the content of the file.


page1 page3

page2 page1

page4 page1

page3 page1

page4 page2

page3 page4

Run the following cells, and complete the cells.

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [11]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=42d0f9b4f4590b290fb149aa1731e46935501d16143ba28b9ee44eddeefe7cf2
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [12]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

FileNotFoundError: ignored

In [7]:
# load the file and persist the RDD in memory
rdd = sc.textFile('network.txt').persist() 

NameError: ignored

In [ ]:
# this is how we compute contribute score for each link.
# for example, if a page's rank is 1.0 and it has 2 outgoing links, each links will have contribution 0.5.
# (neighbors=[page1, page2], rank=1.0)  => ([(page1, 0.5), (page2,0.5)])
def computeContribs(neighbors, rank):
    for neighbor in neighbors:
        yield (neighbor, rank/len(neighbors))

In [ ]:
# TO COMPLETE: for each page, output its outgoing neighbors in a list.
# For example: page3 has two outgoing links to page 1 and page4, the output will be (page3, [page1, page4])
# The content of linksRDD is:
# page4 [u'page1', u'page2']
# page2 [u'page1']
# page3 [u'page1', u'page4']
# page1 [u'page3']

#linksRDD = rdd.flatMap(lambda x:x.split(" ")).map(lambda x:x).reduceByKey(lambda x,y:x+y
linksRDD= rdd.map(lambda x:tuple(x.split(" "))).map(lambda x:(x[0],[x[1]])).reduceByKey(lambda x, y: x+y).collect()
linksRDD


[('page1', ['page3']),
 ('page2', ['page1']),
 ('page4', ['page1', 'page2']),
 ('page3', ['page1', 'page4'])]

In [ ]:
# TO COMPLETE: for each page, initialize its pagerank score to be 1.0
# The content of ranksRDD is:
# [(u'page4', 1.0), (u'page2', 1.0), (u'page3', 1.0), (u'page1', 1.0)]
linksRDD2=sc.parallelize (linksRDD)
ranksRDD=linksRDD2.map(lambda x:(x[0],1.0)).collect()
ranksRDD

[('page1', 1.0), ('page2', 1.0), ('page4', 1.0), ('page3', 1.0)]

In [ ]:
# TO COMPLETE: PageRank is an iterative algorithm. 
# For each iteration, each page's pagerank score will be updated. 
# Refer to lecture slides for method details.
ranksRDD2=sc.parallelize(ranksRDD)
for iteration in range(10): 
  # calculate the contribution of each page's outgoing link
  contribs=linksRDD2.join(ranksRDD2).map(lambda x:x[1]).flatMap(lambda x: computeContribs(x[0],x[1]))
  # update each page's page rank by summing up all incoming link's contribution
  ranksRDD2 =contribs.reduceByKey(lambda x,y:x+y)


In [ ]:
# Return the final page rank score of each page. The results is
# [(u'page4', 0.7109375), (u'page2', 0.375), (u'page3', 1.484375), (u'page1', 1.4296875)]
ranksRDD2.collect()

[('page4', 0.7109375),
 ('page2', 0.375),
 ('page1', 1.4296875),
 ('page3', 1.484375)]